<a href="https://colab.research.google.com/github/JoseAndresEscobar19/Materia_integradora2020I/blob/master/AnalisisCBIS_DDSM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importando las librerias

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
rutadrive="/content/drive/My Drive/Materia_integradora/Datasets/CBIS-DDMS/"
ruta="/content/Dataset/"

In [ ]:
# !rm *.nii.gz
# !rm -r Dataset
!rm -r ddsm_tools/

In [ ]:
!cp -r /content/drive/My\ Drive/Materia_integradora/Datasets/CBIS-DDMS/Librerias/ddsm_tools /content/ddsm_tools
!chmod 755 ddsm_tools/jpegdir/jpeg
!apt-get install -y pigz
!apt-get install dcm2niix
!pip install pydicom
!pip install opencv-python
!pip install pillow
!pip install nipype
!pip install nibabel
 
import ddsm_tools.my_ddsm_converter as ddsm
import pandas as pd 
import os
import zipfile
import numpy as np
import urllib.request, urllib.error, urllib.parse
import urllib.request, urllib.parse, urllib.error
import sys
import math
import tempfile
import pydicom
from pydicom.uid import ImplicitVRLittleEndian
from pydicom.dataset import Dataset, FileDataset
import matplotlib.pyplot as plt
import cv2
import csv
import shutil
import subprocess
import glob
import nibabel as nib
from PIL import Image

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pigz is already the newest version (2.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
dcm2niix is already the newest version (1.0.20171215-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
if not os.path.exists("Dataset"):
  os.mkdir("Dataset")

### Obteniendo estadisticas del dataset

In [ ]:
df_calc_train=pd.read_csv(rutadrive+"calc_case_description_train_set.csv")
df_2=df_calc_train[df_calc_train['pathology'].isin(["MALIGNANT","BENIGN"])]
df_2.head()
print(df_2["pathology"].value_counts())
dff_2 = df_2[["pathology","patient_id"]]
d=dff_2.groupby(['patient_id','pathology']).count().reset_index()
d["pathology"].value_counts()

MALIGNANT    544
BENIGN       528
Name: pathology, dtype: int64


MALIGNANT    276
BENIGN       258
Name: pathology, dtype: int64

In [ ]:
df_calc_test=pd.read_csv(rutadrive+"calc_case_description_test_set.csv")
df_calc_test2=df_calc_test[df_calc_test['pathology'].isin(["MALIGNANT","BENIGN"])]
df_calc_test2.head()
print(df_calc_test2["pathology"].value_counts())
dff_2 = df_calc_test2[["pathology","patient_id"]]
d=dff_2.groupby(['patient_id','pathology']).count().reset_index()
d["pathology"].value_counts()

BENIGN       130
MALIGNANT    129
Name: pathology, dtype: int64


BENIGN       70
MALIGNANT    67
Name: pathology, dtype: int64

In [ ]:
df_masa_train=pd.read_csv(rutadrive+"mass_case_description_train_set.csv")
df_masa_train2=df_masa_train[df_masa_train['pathology'].isin(["MALIGNANT","BENIGN"])]
df_masa_train2.head()
print(df_masa_train2["pathology"].value_counts())
dff_2 = df_masa_train2[["pathology","patient_id"]]
d=dff_2.groupby(['patient_id','pathology']).count().reset_index()
d["pathology"].value_counts()

MALIGNANT    637
BENIGN       577
Name: pathology, dtype: int64


MALIGNANT    346
BENIGN       304
Name: pathology, dtype: int64

In [ ]:
df_masa_test=pd.read_csv(rutadrive+"mass_case_description_test_set.csv")
df_masa_test2=df_masa_test[df_masa_test['pathology'].isin(["MALIGNANT","BENIGN"])]
df_masa_test2.head()
print(df_masa_test2["pathology"].value_counts())
dff_2 = df_masa_test2[["pathology","patient_id"]]
d=dff_2.groupby(['patient_id','pathology']).count().reset_index()
d["pathology"].value_counts()

BENIGN       194
MALIGNANT    147
Name: pathology, dtype: int64


BENIGN       101
MALIGNANT     85
Name: pathology, dtype: int64

### Creando particiones de los datasets

In [ ]:
df_benigno_calc_full=df_calc_train[df_calc_train['pathology'].isin(["BENIGN"])]
df_benigno_calc = df_benigno_calc_full.head(50)
df_maligno_calc_full=df_calc_train[df_calc_train['pathology'].isin(["MALIGNANT"])]
df_maligno_calc = df_maligno_calc_full.head(50)
df_benigno_mass_full=df_masa_train[df_masa_train['pathology'].isin(["BENIGN"])]
df_benigno_mass = df_benigno_mass_full.head(50)
df_maligno_mass_full=df_masa_train[df_masa_train['pathology'].isin(["MALIGNANT"])]
df_maligno_mass = df_maligno_mass_full.head(50)

### Clase para conexion y descarga de imagenes

In [ ]:
#
# Refer https://wiki.cancerimagingarchive.net/display/Public/REST+API+Usage+Guide for complete list of API
#
class TCIAClient:
    GET_IMAGE = "getImage"

    def __init__(self , baseUrl, resource):
        self.baseUrl = baseUrl + "/" + resource

    def execute(self, url, queryParameters={}):
        queryParameters = dict((k, v) for k, v in queryParameters.items() if v)
        queryString = "?%s" % urllib.parse.urlencode(queryParameters)
        requestUrl = url + queryString
        request = urllib.request.Request(url=requestUrl)
        resp = urllib.request.urlopen(request)
        return resp


    def get_image(self , seriesInstanceUid , downloadPath, zipFileName):
        serviceUrl = self.baseUrl + "/query/" + self.GET_IMAGE
        queryParameters = { "SeriesInstanceUID" : seriesInstanceUid }
        os.umask(0o002)
        try:
            file = os.path.join(downloadPath, zipFileName)
            resp = self.execute( serviceUrl , queryParameters)
            downloaded = 0
            CHUNK = 256 * 10240
            with open(file, 'wb') as fp:
                while True:
                    chunk = resp.read(CHUNK)
                    downloaded += len(chunk)
                    if not chunk: break
                    fp.write(chunk)
        except urllib.error.HTTPError as e:
            print("HTTP Error:",e.code , serviceUrl)
            return False
        except urllib.error.URLError as e:
            print("URL Error:",e.reason , serviceUrl)
            return False

        return True

In [ ]:
tcia_client = TCIAClient(baseUrl="https://services.cancerimagingarchive.net/services/v4",resource = "TCIA")

## Descargando las imagenes y convirtiendolas a nii

In [ ]:
def descargar_imagenes_imgs_2(tcia,data):
  datos={"patient_id":[],"abnormality type":[],"image_path":[],"cropped_path":[],"roi_path":[]}
  for imgpath,roipath,croppath,pathology,abnormality in zip(data["image file path"],
                                                data["cropped image file path"],
                                                data["ROI mask file path"],
                                                data["pathology"],
                                                data["abnormality type"]): 
    print(".",end="")
    imgpath=imgpath.strip()
    roipath=roipath.strip()
    croppath=croppath.strip()
    # obtiniendo informacion de la imagen
    patient_id,StudyInstanceUD1,SeriesInstanceUID1,filename=imgpath.rstrip().split("/")
    patient_id2,StudyInstanceUD2,SeriesInstanceUID2,filename2=roipath.rstrip().split("/")
    patient_id3,StudyInstanceUD3,SeriesInstanceUID3,filename3=croppath.rstrip().split("/")
 
    ruta_full=os.path.join(pathology,"full_mammo",patient_id,SeriesInstanceUID1)
    ruta_roi=os.path.join(pathology,"ground_truth",patient_id,SeriesInstanceUID2)
 
    datos["abnormality type"].append(abnormality)
    datos["patient_id"].append(patient_id)
    # Si la ruta no existe, se la crea (es temporal)
    if not os.path.exists(ruta+ruta_full):
      os.makedirs(ruta+ruta_full)
    if not os.path.exists(rutadrive+ruta_full):
      os.makedirs(rutadrive+ruta_full)
    if not os.path.exists(rutadrive+ruta_roi):
      os.makedirs(rutadrive+ruta_roi)
 
    # descargando las imagenes del repositorio CBIS
    descargadofull=tcia.get_image(SeriesInstanceUID1,ruta,"images.zip");
 
    #descomprimiendo las imagenes y renombrarlas a un nombre mas significativo
    if descargadofull:
      with zipfile.ZipFile(ruta+"images.zip", 'r') as zip_ref:
        member=zip_ref.namelist()[1]
        zip_ref.extract(member,ruta+ruta_full)
        os.rename(os.path.join(ruta,ruta_full,member),os.path.join(ruta,ruta_full,"full_mammo.dcm"))
        datos["image_path"].append(os.path.join(ruta_full,"full_mammo.dcm"))
    else:
      print(ruta_full+member)
 
    descargadoroicrop=tcia.get_image(SeriesInstanceUID2,ruta,"images.zip");
    if roipath[:-11]==croppath[:-11]:
      if descargadoroicrop:
        with zipfile.ZipFile(ruta+"images.zip", 'r') as zip_ref:
          member_1,member_2= zip_ref.namelist()[1:]
          zip_ref.extract(member_1,ruta+ruta_roi)
          zip_ref.extract(member_2,ruta+ruta_roi)

          d=pydicom.filereader.dcmread(os.path.join(ruta,ruta_roi,member_1)).SeriesDescription
          p1=os.path.join(ruta_roi,"crop_mammo.dcm")
          p2=os.path.join(ruta_roi,"roi_full_mammo.dcm")
          datos["cropped_path"].append(p1)
          datos["roi_path"].append(p2)
          # cropped images
          # ROI mask images
          if d == "ROI mask images":
            p1=os.path.join(ruta_roi,"roi_full_mammo.dcm")
            p2=os.path.join(ruta_roi,"crop_mammo.dcm")
          os.rename(os.path.join(ruta,ruta_roi,member_1),ruta+p1)
          os.rename(os.path.join(ruta,ruta_roi,member_2),ruta+p2)
 
    else:
      descargadocrop=tcia.get_image(SeriesInstanceUID3,ruta,"images2.zip")
      if descargadoroicrop and descargadocrop:
        with zipfile.ZipFile(ruta+"images.zip", 'r') as zip_ref:
          member= zip_ref.namelist()[1]
          zip_ref.extract(member,ruta+ruta_roi)
          os.rename(os.path.join(ruta,ruta_roi,member),os.path.join(ruta,ruta_roi,"roi_full_mammo.dcm"))
          datos["roi_path"].append(os.path.join(ruta_roi,"roi_full_mammo.dcm"))
        with zipfile.ZipFile(ruta+"images2.zip", 'r') as zip_ref:
          member= zip_ref.namelist()[1]
          zip_ref.extract(member,ruta+ruta_roi)
          os.rename(os.path.join(ruta,ruta_roi,member),os.path.join(ruta,ruta_roi,"crop_mammo.dcm"))
          datos["cropped_path"].append(os.path.join(ruta_roi,"crop_mammo.dcm"))
  print()
  return datos

In [ ]:
def do_convert_dicom_to_nii(datos):
  newdatos={}
  for tipo in datos:
    if tipo not in newdatos:
      newdatos[tipo]=[]
    for img in datos[tipo]:
      if ".dcm" not in img:
        newdatos[tipo].append(img)
      else:
        pathisto=rutadrive+img[:img.rfind("/")]+img[img.rfind("/"):-4]+".nii.gz"
        if not os.path.exists(pathisto):
          cmd=["dcm2niix","5","-b","n","-s","y","-z","y","-f",img[img.rfind("/")+1:-4],"-o",rutadrive+img[:img.rfind("/")+1],ruta+img]
          r=subprocess.run(cmd)
          if r.returncode!=0:
            print("Problema con "+img,r)
          #else: os.remove(ruta+img)
        newdatos[tipo].append(pathisto)
  return newdatos

In [ ]:
# datos1=descargar_imagenes_imgs_2(tcia_client,df_benigno_calc_full[:1])
# dic_benigno_calc=do_convert_dicom_to_nii(datos1)

.


In [ ]:
# datos2=descargar_imagenes_imgs_2(tcia_client,df_benigno_mass_full[50:df_benigno_mass_full.shape[0]//2])
# dic_benigno_mass=do_convert_dicom_to_nii(datos2)

In [ ]:
# datos3=descargar_imagenes_imgs_2(tcia_client,df_maligno_calc_full[50:df_maligno_calc_full.shape[0]//2])
# dic_maligno_calc=do_convert_dicom_to_nii(datos3)

In [ ]:
# datos4=descargar_imagenes_imgs_2(tcia_client,df_maligno_mass_full[50:df_maligno_mass_full.shape[0]//2])
# dic_maligno_mass=do_convert_dicom_to_nii(datos4)

## csv Creados de Benigno y Maligno

In [ ]:
#CAMBIAR NOMBRES SEGUN LOS DF QUE SE USEN
nombresdf=["malignant_calc_case_description_train_set_new.csv",
           "malignant_mass_case_description_train_set_new.csv",
           "benign_calc_case_description_train_set_new.csv",
           "benign_mass_case_description_train_set_new.csv"]

In [ ]:
df_mlg_mass.drop(df_mlg_mass[df_mlg_mass.patient_id.isin(["Mass-Training_P_00370_RIGHT_MLO","Mass-Training_P_00406_RIGHT_MLO","Mass-Training_P_00637_RIGHT_CC","Mass-Training_P_00720_RIGHT_MLO","Mass-Training_P_00637_RIGHT_CC"])].index,axis=0,inplace=True)#.reset_index(inplace=True)
df_mlg_mass.reset_index(drop=True,inplace=True)
df_mlg_mass.shape

(313, 5)

In [ ]:
# df_bng_calc=pd.DataFrame(dic_benigno_calc)
# df_bng_calc.to_csv(rutadrive+nombresdf[2])

# df_bng_mass=pd.DataFrame(dic_benigno_mass)
# df_bng_mass.to_csv(rutadrive+nombresdf[3])
 
# df_mlg_calc1=pd.DataFrame(dic_maligno_calc)
# df_mlg_calc.to_csv(rutadrive+nombresdf[0])
 
# df_mlg_mass=pd.DataFrame(dic_maligno_mass)
# df_mlg_mass.to_csv(rutadrive+nombresdf[1])

In [ ]:
df_mlg_calc=pd.read_csv(rutadrive+nombresdf[0],index_col=0)

df_mlg_mass=pd.read_csv(rutadrive+nombresdf[1],index_col=0)

df_bng_calc=pd.read_csv(rutadrive+nombresdf[2],index_col=0)

df_bng_mass=pd.read_csv(rutadrive+nombresdf[3],index_col=0)

In [ ]:
print(df_mlg_calc.shape)
print(df_mlg_mass.shape)
print(df_bng_calc.shape)
print(df_bng_mass.shape)

(271, 5)
(313, 5)
(263, 5)
(288, 5)


## Obteniendo imagenes normales

In [ ]:
from ftplib import FTP
import os
import urllib.request as req
import shutil
from contextlib import closing
 
def dowload_normales():
  n=0
  host="figment.csee.usf.edu"
  folder = ruta + 'NORMAL/'
  endrive = rutadrive+'NORMAL/'
  if not os.path.exists(folder):
    os.makedirs(folder)
  lista=[]
  with FTP(host,timeout=30) as ftp:
      if not os.path.exists(folder):
          os.makedirs(folder)
      if not os.path.exists(endrive):
          os.makedirs(endrive)
      # Login to FTP Server
      ftp.login()
      normals="pub/DDSM/cases/normals"
      # Get the list of files in the server
      lista_normales = ftp.nlst(normals)
      for dir in lista_normales:
        # Change the working directory
        ftp.cwd("/"+dir+"/")
        lista_casos=ftp.nlst()
        for caso in lista_casos: 
          if n>150:
            return lista
          print(caso,n)
          lista.append("NORMAL/"+caso)
          n+=1
          if os.path.exists(endrive+caso+"/"):
            if len(os.listdir(endrive+caso+"/"))>=5:
              continue
          else: 
            os.makedirs(endrive+caso+"/")
 
          if os.path.exists(folder+caso+"/"):
            if len(os.listdir(folder+caso+"/"))>=5:
              continue
          else: 
            os.makedirs(folder+caso+"/")
          
          ftp.cwd("/"+dir+"/"+caso+"/")   
          for filename in ftp.nlst():
            if ".16_PGM" in filename:
              continue
            # Create the URL from where file has to be downloaded
            file = 'ftp://{}/{}/{}/{}'.format(host,dir,caso,filename)
            # Create the file name
            download_file_name = folder + caso+"/"+filename
            with closing(req.urlopen(file)) as r:
                with open(download_file_name, 'wb') as f:
                    shutil.copyfileobj(r, f)
  return lista

In [ ]:
prueba=dowload_normales()

case0002 0
case0003 1
case0005 2
case0006 3
case0009 4
case0010 5
case0011 6
case0012 7
case0013 8
case0014 9
case0015 10
case0016 11
case0018 12
case0020 13
case0022 14
case0024 15
case0026 16
case0027 17
case0030 18
case0031 19
case0032 20
case0034 21
case0035 22
case0036 23
case0037 24
case0039 25
case0040 26
case0041 27
case0042 28
case0043 29
case0044 30
case0045 31
case0046 32
case0048 33
case0049 34
case0050 35
case0051 36
case0052 37
case0053 38
case0055 39
case0056 40
case0057 41
case0058 42
case0059 43
case0061 44
case0062 45
case0063 46
case0064 47
case0067 48
case0068 49
case0069 50
case0070 51
case0071 52
case0072 53
case0073 54
case0074 55
case0075 56
case0076 57
case0077 58
case0079 59
case0080 60
case0089 61
case0090 62
case0091 63
case0092 64
case0093 65
case0094 66
case0095 67
case0096 68
case0097 69
case0099 70
case0108 71
case0109 72
case0110 73
case0111 74
case0112 75
case0114 76
case0115 77
case0116 78
case0119 79
case0121 80
case0123 81
case0124 82
case0126 83
ca

In [ ]:
def split_filename(filepath):
  path = os.path.dirname(filepath)
  filename = os.path.basename(filepath)
  base, ext = os.path.splitext(filename)
  return path, base, ext

In [ ]:
#IMAGENES TIFF A DICOM
def convert_tiff_dicom(ics_dict,tiff,force=False):
  """
  :param ics_dict: dict with infomation of the image
  :param tif: path to tif image
  :param force: if True, force the creation of the file
  :return: converted file path
  """
  path,base,ext=split_filename(tiff)
  dicom_filename=os.path.join(base+".dcm")
 
  if os.path.exists(path+"/"+dicom_filename) and not force:
    return (base,dicom_filename)
  _,gg=base.split(".")
  # Create the metadata for the dataset
  file_meta = Dataset()
  #file_meta.TransferSyntaxUID = ImplicitVRLittleEndian
 
  # Create some temporary filenames
  suffix = '.dcm'
  filename_little_endian = tempfile.NamedTemporaryFile(suffix=suffix).name
  filename_big_endian = tempfile.NamedTemporaryFile(suffix=suffix).name
 
  # CONFIGURACION DE IMAGEN DICOM BASE CBIS DDSM
  file_meta.MediaStorageSOPClassUID = 'Secondary Capture Image Storage'
  file_meta.MediaStorageSOPInstanceUID = '1.3.6.1.4.1.9590.100.1.2.289923739312470966435676008311959891294'
  file_meta.ImplementationClassUID = '1.2.40.0.13.1.1.1'
  
  # Create the dataset
  ds = FileDataset(filename_little_endian, {}, file_meta=file_meta, preamble=b'\x00'*128) # pre-existing dicom file
  ds.is_little_endian = True
  ds.is_implicit_VR = True
 
  ds.PatientName = ics_dict["patient_id"]
  ds.PatientID = ics_dict["patient_id"]
  im_frame = Image.open(tiff)
  np_frame = np.array(im_frame.getdata(),dtype=np.uint16)
  ds.Rows = ics_dict[gg]["height"]
  ds.Columns = ics_dict[gg]["width"]
  ds.PhotometricInterpretation = "MONOCHROME2"
  ds.BitsAllocated = ics_dict[gg]["bpp"]
  ds.BitsStored = ics_dict[gg]["bpp"]
  ds.SmallestImagePixelValue = 0
  ds.LargestImagePixelValue  = 65535
  ds.HighBit = 15
  ds.PixelRepresentation = 0    #fijo, todos ejemplos usan 0
  ds.PixelData = np_frame.tobytes()
  ds.save_as(path+"/"+dicom_filename)
  return (base,dicom_filename)

In [ ]:
def convert_LJPEG_dcm(dirs):
  datos={"patient_id":[],"image_path":[]}
  for dir in dirs:
    try:
      ics = glob.glob(ruta+dir + '/*.ics')[0]
    except:
      continue
    ics_dict=ddsm.get_ics_info(ics)
   
    for f in os.listdir(ruta+dir):
      
      if ".LJPEG" not in f:
        continue
      
      normal = ddsm.ddsm_abnormality(os.path.join(ruta,dir,f),ics_dict)
      tif_file=normal.save_image(od_correct=True)
      patient_id,dcm_file=convert_tiff_dicom(ics_dict,tif_file)      
      datos["patient_id"].append(patient_id)
      datos["image_path"].append(os.path.join(dir,dcm_file))
 
  return datos

In [ ]:
#dcms=convert_LJPEG_dcm(prueba)

In [ ]:
#normales=do_convert_dicom_to_nii(dcms)

In [ ]:
#df_normals=pd.DataFrame(normales)

(604, 2)

In [ ]:
#df_normals.to_csv(rutadrive+"normal_case_set.csv")

In [ ]:
df_normals=pd.read_csv(rutadrive+"normal_case_set.csv",index_col=0)